### 实例描述
#### 通过一个带有全局平均池化层的卷积神经网络对CIFAR数据集分类

### 1.导入头文件引入数据集

In [3]:
from cifar10 import cifar10_input
import tensorflow as tf
import numpy as np

In [4]:
# 读数据
batch_size = 128
# data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'
print("begin")
images_train, labels_train = cifar10_input.inputs(eval_data = False, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, batch_size = batch_size)
print("begin data")

begin
begin data


### 2.定义网络结构

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层，步长为1
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 最大池化层，步长2, 2*2窗口
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  

# 最大池化层，步长6, 6*6窗口
def avg_pool_6x6(x):
    return tf.nn.avg_pool(x, ksize=[1, 6, 6, 1], strides=[1, 6, 6, 1], padding='SAME')

In [6]:
# 定义占位符
x = tf.placeholder(tf.float32, [None, 24,24,3]) # cifar data image of shape 24*24*3
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 数字=> 10 classes

In [7]:
W_conv1 = weight_variable([5, 5, 3, 64])
b_conv1 = bias_variable([64])

x_image = tf.reshape(x, [-1,24,24,3])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 64, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([5, 5, 64, 10])
b_conv3 = bias_variable([10])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)

nt_hpool3=avg_pool_6x6(h_conv3)#10
nt_hpool3_flat = tf.reshape(nt_hpool3, [-1, 10])
# softmax函数
y_conv=tf.nn.softmax(nt_hpool3_flat)

cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

### 3.运行session进行训练

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# tf.train.start_queue_runners(sess=sess)
for i in range(15000):#20000
    image_batch, label_batch = sess.run([images_train, labels_train])
    label_b = np.eye(10,dtype=float)[label_batch] #one hot
  
    train_step.run(feed_dict={x:image_batch, y: label_b},session=sess)
  
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:image_batch, y: label_b},session=sess)
        print( "step %d, training accuracy %g"%(i, train_accuracy))

step 0, training accuracy 0.09375
step 200, training accuracy 0.273438
step 400, training accuracy 0.375
step 600, training accuracy 0.484375
step 800, training accuracy 0.40625
step 1000, training accuracy 0.46875
step 1200, training accuracy 0.539062
step 1400, training accuracy 0.53125
step 1600, training accuracy 0.515625
step 1800, training accuracy 0.5
step 2000, training accuracy 0.53125
step 2200, training accuracy 0.507812
step 2400, training accuracy 0.617188
step 2600, training accuracy 0.578125
step 2800, training accuracy 0.585938
step 3000, training accuracy 0.523438
step 3200, training accuracy 0.601562
step 3400, training accuracy 0.585938
step 3600, training accuracy 0.585938
step 3800, training accuracy 0.640625
step 4000, training accuracy 0.507812
step 4200, training accuracy 0.492188
step 4400, training accuracy 0.53125
step 4600, training accuracy 0.65625
step 4800, training accuracy 0.632812
step 5000, training accuracy 0.507812
step 5200, training accuracy 0.546

### 4.评估结果

In [9]:
image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch] #one hot编码
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={x:image_batch, y: label_b},session=sess))

finished！ test accuracy 0.648438
